# Buses Compare

In [1]:
import pandas as pd
import folium

df_buses_385 = pd.read_csv('buses_385.csv')
df_buses_501 = pd.read_csv('buses_501.csv')

df_lines = pd.read_csv('lines_501.csv')

x = dict(zip(df_buses_501['bus_id'], df_buses_501['x']))
y = dict(zip(df_buses_501['bus_id'], df_buses_501['y']))

In [2]:
map_GB = folium.Map(location=[54, -2], zoom_start=6, tiles="CartoDB positron")

for _, row in df_buses_385.iterrows():
    
    popup_html = f"""
    <b>Bus ID:</b> {row.get("buses_id", "")}<br>
    <b>x:</b> {row.get("x", "")}<br>
    <b>y:</b> {row.get("y", "")}
    """

    folium.CircleMarker(
        location=[row["y"], row["x"]],
        radius=2,
        color="green",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=200)
    ).add_to(map_GB)

In [3]:
for _, row in df_buses_501.iterrows():

    popup_html = f"""
    <b>Bus ID:</b> {row.get("bus_id", "")}<br>
    <b>x:</b> {row.get("x", "")}<br>
    <b>y:</b> {row.get("y", "")}
    """

    folium.CircleMarker(
        location=[row["y"], row["x"]],
        radius=2,
        color="red",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=200)
    ).add_to(map_GB)

## Add units(generators)

In [4]:
df_units_capacity = pd.read_csv('../../GBPower_raw/bmus_prepared.csv')
df_units = pd.read_csv('../../GBPower_raw/prepared_bmus.csv')

df_units['capacity'] = df_units_capacity['capacity']
df_units

,NationalGridBmUnit,carrier,bus,lon,lat,capacity
0,IBG-EGEN1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
1,IBG-ENGI1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
2,IBG-RSQ1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
3,IVD-VKL1,interconnector,7909,-0.221,52.931,0.0
4,IVG-VKL1,interconnector,7909,-0.221,52.931,0.0
...,...,...,...,...,...,...
1060,SFERD-1,demand_flex,distributed,distributed,distributed,240.0
1061,EAS-SEL01,demand_flex,distributed,distributed,distributed,0.0
1062,SEE-PGU01,demand_flex,distributed,distributed,distributed,0.0
1063,YEP-ATL01,demand_flex,distributed,distributed,distributed,0.0


In [5]:
for _, row in df_units.iterrows():
    if row.get("lon") == 'distributed':
        continue
    popup_html = f"""
    <b>Unit:</b> {row.get("NationalGridBmUnit", "")}<br>
    <b>x:</b> {row.get("lon", "")}<br>
    <b>y:</b> {row.get("lat", "")}<br>
    <b>Capacity:</b> {row.get("capacity", "")}
    """

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=2,
        color="purple",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=200)
    ).add_to(map_GB)

# Add Lines between buses

In [6]:
for _, row in df_lines.iterrows():
    bus0, bus1 = row['bus0'], row['bus1']
    
    try:
        coord0 = (y[bus0], x[bus0])
        coord1 = (y[bus1], x[bus1])

        folium.PolyLine(
            locations=[coord0, coord1],
            color='blue',
            weight=1.2,
            opacity=0.8,
            popup=f"{bus0} → {bus1}"
        ).add_to(map_GB)
    
    except KeyError as e:
        print(f"Missing bus in coordinate mapping: {e}")
        continue


Missing bus in coordinate mapping: 'way/264275258-275'
Missing bus in coordinate mapping: 'way/264275258-275'


## Add Lines between buses and generators

In [121]:
x_385 = dict(zip(df_buses_385['buses_id'].astype(str), df_buses_385['x']))
y_385 = dict(zip(df_buses_385['buses_id'].astype(str), df_buses_385['y']))

for _, row in df_units.iterrows():
    gen, bus = row['NationalGridBmUnit'], row['bus']
    
    try: 
        coord0 = (y_385[bus], x_385[bus])
        coord1 = (row['lat'], row['lon'])
        
        folium.PolyLine(
            locations=[coord0, coord1],
            color='black',
            weight=0.8,
            opacity=0.4,
            popup=f"{gen} → {bus}"
        ).add_to(map_GB)
    
    except KeyError as e:
        print(f"Missing bus in coordinate mapping: {e}")
        continue

Missing bus in coordinate mapping: 'distributed'
Missing bus in coordinate mapping: 'distributed'
Missing bus in coordinate mapping: 'distributed'
Missing bus in coordinate mapping: 'distributed'
Missing bus in coordinate mapping: 'distributed'
Missing bus in coordinate mapping: 'distributed'
Missing bus in coordinate mapping: 'distributed'
Missing bus in coordinate mapping: 'distributed'
Missing bus in coordinate mapping: 'distributed'


## Save the figure

In [122]:
map_GB.save("buses_compare.html")